In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/ccdata/CC GENERAL.csv')
df

In [ ]:
df.describe()


In [ ]:
df[df['ONEOFF_PURCHASES'] == 40761.25]

In [ ]:
cash_adv_max = df.CASH_ADVANCE.max()

In [ ]:
df[df['CASH_ADVANCE'] == cash_adv_max]

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar= False, cmap='Blues')

In [ ]:
df.fillna(df.mean(), inplace= True)

In [ ]:
sns.heatmap(df.isnull(), yticklabels= False, cbar=False, cmap='Blues')

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop('CUST_ID', axis = 1, inplace= True)

In [ ]:
df

In [ ]:
len(df.columns)

In [ ]:
correlations = df.corr();

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(correlations, annot=True);

In [ ]:
# Let's scale the data first
scaler = StandardScaler();
df_scaled = scaler.fit_transform(df)

In [ ]:
df_scaled.shape

In [ ]:
# KMeans Clustering Elbow Method
scores = []

ranges = range(1, 20)

for i in ranges:
  kmeans = KMeans(n_clusters = i)
  kmeans.fit(df_scaled)
  scores.append(kmeans.inertia_) 

plt.plot(scores, 'bx-')
plt.title('Finding the right number of clusters')
plt.xlabel('Clusters')
plt.ylabel('Scores')
plt.show()

In [ ]:
kmeans = KMeans(8)
kmeans.fit(df_scaled)
labels = kmeans.labels_

In [ ]:
kmeans.cluster_centers_.shape

In [ ]:
clusters_centers = pd.DataFrame(data = kmeans.cluster_centers_, columns= [df.columns])
clusters_centers

In [ ]:
labels.shape

In [ ]:
labels.max(), labels.min()

In [ ]:
y_kmeans = kmeans.fit_predict(df_scaled)
y_kmeans

In [ ]:
# concatenate the clusters labels to our original dataframe

df_clusters = pd.concat([df, pd.DataFrame({'cluster':labels})], axis=1)
df_clusters.head()

In [ ]:
# Plot the histogram of various clusters
for i in df.columns:
  plt.figure(figsize=(35,5))
  for j in range(8):
    plt.subplot(1,8, j+1)
    cluster = df_clusters[df_clusters['cluster'] == j]
    cluster[i].hist(bins=30)
    plt.title('{} \ncluster {} '.format(i, j))

  plt.show()

In [ ]:
# principal components analysis
pca = PCA(n_components=2)
principal_components = pca.fit_transform(df_scaled)
principal_components

In [ ]:
pca_df = pd.DataFrame(data = principal_components, columns=['pca1', 'pca2'])
pca_df

In [ ]:
pca_df = pd.concat([pca_df, pd.DataFrame({'cluster': labels})], axis= 1)
pca_df.head()

In [ ]:
plt.figure(figsize=(20,20))
sns.scatterplot(data = pca_df, x = 'pca1', y= 'pca2', hue = 'cluster', palette =['red','green','blue','pink','yellow','gray','purple', 'black']);

In [ ]:
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from keras.optimizers import SGD

In [ ]:
input_df = Input(shape=(17,))

X = Dense(7, activation='relu')(input_df)
X = Dense(500, activation='relu', kernel_initializer= 'glorot_uniform')(X)
X = Dense(500, activation='relu', kernel_initializer= 'glorot_uniform')(X)
X = Dense(500, activation='relu', kernel_initializer= 'glorot_uniform')(X)
X = Dense(2000, activation='relu', kernel_initializer= 'glorot_uniform')(X)

encoded = Dense(10, activation='relu', kernel_initializer='glorot_uniform')(X)
X = Dense(2000, activation='relu', kernel_initializer= 'glorot_uniform')(encoded)
X = Dense(500, activation='relu', kernel_initializer= 'glorot_uniform')(X)

decoded = Dense(17, activation='relu', kernel_initializer='glorot_uniform')(X)


autoendocer = Model(input_df, decoded)

encoder = Model(input_df, encoded)

autoendocer.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
autoendocer.fit(df_scaled, df_scaled, batch_size=128, epochs=25, verbose= 1)

In [ ]:
pred = encoder.predict(df_scaled)

In [ ]:
pred.shape

In [ ]:
scores_2 = []

range_values = range(1, 20)

for i in range_values:
  kmeans = KMeans(n_clusters = i)
  kmeans.fit(pred)
  scores_2.append(kmeans.inertia_) 

plt.plot(scores_2, 'bx-')
plt.title('Finding the right number of clusters')
plt.xlabel('Clusters')
plt.ylabel('Scores') 
plt.show()

In [ ]:
plt.plot(scores, 'bx-', color ='r')
plt.plot(scores_2, 'bx-', color ='g')

In [ ]:
kmeans = KMeans(4)
kmeans.fit(pred)
labels = kmeans.labels_

In [ ]:
df_clusters = pd.concat([df, pd.DataFrame({'cluster': labels})], axis=1)
df_clusters.head()

In [ ]:
labels

In [ ]:
pca = PCA(n_components=2)
pca_comp = pca.fit_transform(pred)
pca_df = pd.DataFrame(data = pca_comp, columns=['pca1' , 'pca2'])
pca_df.head()

In [ ]:
pca_df = pd.concat([pca_df, pd.DataFrame({'cluster': labels})], axis=1)
pca_df.head()

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x='pca1', y = 'pca2', data = pca_df, hue='cluster', palette=['red', 'green', 'blue', 'yellow'])